# Totale

##### setup

In [12]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")



dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(',','.')


Username:  davidguzman


# Ricavi

In [13]:
dfClienti

,Unnamed: 0,Nr,CodCondizioniPagam,FattCumulative,Valuta
0,0,C00001,101,False,1
1,1,C00003,102,False,2
2,2,C00004,102,False,3
3,3,C00005,112,False,1
4,4,C00006,101,False,2
...,...,...,...,...,...
103,103,C00512,100,False,2
104,104,C00513,101,False,3
105,105,C00514,101,False,1
106,106,C00515,101,False,2


In [14]:
dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
venditeConsuntivo = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")
venditeBudget = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

In [22]:
# join with dfCambio and dfClienti
venditeConsuntivo = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeConsuntivo as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Consuntivo' or k.Anno = 'CONSUNTIVO'
''')

venditeBudget = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeBudget as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Budget' or k.Anno = 'BUDGET'
''')

In [23]:
venditeBudget

,NrMovimento,budget,NrArticolo,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,TassoCambioMedio,TotaleEuro,NrOrigine,CodCondizioniPagam,FattCumulative,Valuta
0,24907,BUDGET,ART0000893,2,250.0,1,250.000000,C00001,101,0,1
1,31922,BUDGET,ART0005136,1,250.0,1,250.000000,C00001,101,0,1
2,34528,BUDGET,ART0000877,4,952.0,1,952.000000,C00001,101,0,1
3,34529,BUDGET,ART0000878,4,912.0,1,912.000000,C00001,101,0,1
4,34530,BUDGET,ART0000879,4,312.0,1,312.000000,C00001,101,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1204,29158,BUDGET,ART0004099,1,905.0,123.4,7.333874,C00297,100,0,3
1205,30210,BUDGET,ART0005136,1,120.0,123.4,0.972447,C00297,100,0,3
1206,31893,BUDGET,ART0005136,1,150.0,123.4,1.215559,C00297,100,0,3
1207,35231,BUDGET,ART0004104,4,3520.0,123.4,28.525122,C00297,100,0,3


In [24]:
sumVenditeConsuntivo = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeConsuntivo')
sumVenditeBudget = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeBudget')
sumVenditeBudget-sumVenditeConsuntivo

,sumVenditaTotale
0,-112506.95897
